In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import json

# ==== Wczytywanie pliku CSV ====
glucose_data = pd.read_csv('Michał._glucose_8-3-2025.csv', skiprows=1, delimiter=',', low_memory=False)
glucose_data['Device Timestamp'] = pd.to_datetime(glucose_data['Device Timestamp'], format='%d-%m-%Y %H:%M', errors='coerce')
glucose_data = glucose_data.dropna(subset=['Device Timestamp'])
glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(glucose_data['Scan Glucose mg/dL'])
glucose_data = glucose_data.rename(columns={'Device Timestamp': 'date'})
glucose_data = glucose_data[['date', 'glucose']]

# ==== Wczytywanie pliku JSON ====
with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator:
            glucose = calculator.get('glucose')
            insulin = calculator.get('units', {}).get('short')
            date = pd.to_datetime(calculator.get('date'), errors='coerce')
            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))
            processed_data.append([date, glucose, insulin, carbs])

meals_df = pd.DataFrame(processed_data, columns=['date', 'glucose', 'insulin', 'carbs'])
meals_df['date'] = meals_df['date'].dt.tz_convert(None)

# ==== Dopasowywanie danych ====
glucose_data = glucose_data.sort_values('date').reset_index(drop=True)
meals_df = meals_df.sort_values('date').reset_index(drop=True)

merged_data = []
for _, row in meals_df.iterrows():
    meal_time = row['date']
    glucose_window = glucose_data[(glucose_data['date'] >= meal_time + pd.Timedelta(hours=1)) &
                                  (glucose_data['date'] <= meal_time + pd.Timedelta(hours=2.5))]
    if not glucose_window.empty:
        glucose_max = np.nanmax(glucose_window['glucose'].values)
        glucose_min = np.nanmin(glucose_window['glucose'].values)

        WW = row['carbs'] / 10
        insulin_per_WW = row['insulin'] / WW if WW > 0 else 0
        hour = meal_time.hour + meal_time.minute / 60
        glucose_diff_max = glucose_max - row['glucose']
        glucose_diff_min = glucose_min - row['glucose']

        # ✅ Klasyfikacja dawek
        if glucose_max > 180:
            dose_feedback = 2  # Znacznie za mała dawka
        elif glucose_max > 140:
            dose_feedback = 1  # Za mała dawka
        elif glucose_min < 80:
            dose_feedback = -2  # Znacznie za duża dawka
        elif glucose_min < 100:
            dose_feedback = -1  # Za duża dawka
        else:
            dose_feedback = 0  # OK

        merged_data.append([
            row['glucose'], row['insulin'], WW, hour,
            glucose_diff_max, glucose_diff_min, dose_feedback
        ])

data = pd.DataFrame(merged_data, columns=[
    'glucose', 'insulin', 'WW', 'hour', 'glucose_diff_max', 'glucose_diff_min', 'dose_feedback'
])

# ==== Skalowanie danych ====
scalers = {}
for col in data.columns[:-1]:
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[[col]])
    scalers[col] = scaler

# ==== Klasyfikacja ====
X = data.iloc[:, :-1]
y = data['dose_feedback']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Model RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# ✅ Ocena modelu
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# ==== TESTY ====
test_input = pd.DataFrame([[150, 20, 5, 9.5, 20, -10]], columns=data.columns[:-1])
for col in test_input.columns:
    test_input[col] = scalers[col].transform(test_input[[col]])

predicted_class = model.predict(test_input)[0]

# ✅ Interpretacja wyniku
if predicted_class == 2:
    print("🔥 Dawka znacznie za mała → Zwiększ dawkę insuliny!")
elif predicted_class == 1:
    print("🔥 Dawka za mała → Zwiększ dawkę insuliny!")
elif predicted_class == -2:
    print("❄️ Dawka znacznie za duża → Zmniejsz dawkę insuliny!")
elif predicted_class == -1:
    print("❄️ Dawka za duża → Zmniejsz dawkę insuliny!")
else:
    print("✅ Dawka OK → Kontynuuj w tej samej dawce.")

              precision    recall  f1-score   support

          -2       1.00      1.00      1.00         1
          -1       0.00      0.00      0.00         2
           0       0.00      0.00      0.00         1
           1       0.40      0.80      0.53         5
           2       0.75      0.60      0.67        10

    accuracy                           0.58        19
   macro avg       0.43      0.48      0.44        19
weighted avg       0.55      0.58      0.54        19

🔥 Dawka za mała → Zwiększ dawkę insuliny!


C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classificati

In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report
import json

# ==== Wczytywanie pliku CSV ====
glucose_data = pd.read_csv('Michał._glucose_8-3-2025.csv', skiprows=1, delimiter=',', low_memory=False)

# Konwersja daty na format datetime
glucose_data['Device Timestamp'] = pd.to_datetime(glucose_data['Device Timestamp'], format='%d-%m-%Y %H:%M', errors='coerce')

# Usuwamy błędne daty
glucose_data = glucose_data.dropna(subset=['Device Timestamp'])

# ✅ Łączymy dane z dwóch kolumn (Historic + Scan)
glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(glucose_data['Scan Glucose mg/dL'])

glucose_data = glucose_data.rename(columns={'Device Timestamp': 'date'})
glucose_data = glucose_data[['date', 'glucose']]

# ==== Wczytywanie pliku JSON ====
with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

# Przetwarzanie danych JSON
processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator:
            glucose = calculator.get('glucose')
            insulin = calculator.get('units', {}).get('short')
            date = pd.to_datetime(calculator.get('date'), errors='coerce')
            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))
            processed_data.append([date, glucose, insulin, carbs])

meals_df = pd.DataFrame(processed_data, columns=['date', 'glucose', 'insulin', 'carbs'])
meals_df['date'] = meals_df['date'].dt.tz_convert(None)

# ==== Dopasowywanie danych ====
glucose_data = glucose_data.sort_values('date').reset_index(drop=True)
meals_df = meals_df.sort_values('date').reset_index(drop=True)

merged_data = []
for index, row in meals_df.iterrows():
    meal_time = row['date']
    glucose_window = glucose_data[(glucose_data['date'] >= meal_time + pd.Timedelta(hours=1)) & (glucose_data['date'] <= meal_time + pd.Timedelta(hours=2.5))]

    if not glucose_window.empty:
        glucose_max = np.nanmax(glucose_window['glucose'].values)
        glucose_min = np.nanmin(glucose_window['glucose'].values)

        WW = row['carbs'] / 10
        insulin_per_WW = row['insulin'] / WW if WW > 0 else 0

        hour = meal_time.hour + meal_time.minute / 60
        glucose_diff_max = glucose_max - row['glucose']
        glucose_diff_min = glucose_min - row['glucose']

        # ✅ Nowa logika klasyfikacji
        if (80 <= row['glucose'] + glucose_diff_max <= 150) and (80 <= row['glucose'] + glucose_diff_min <= 150):
            dose_feedback = 0  # OK
        elif abs(glucose_diff_max) <= 30 and abs(glucose_diff_min) <= 30:
            dose_feedback = 0  # OK
        elif glucose_diff_max > 100:
            dose_feedback = 2  # Znacznie za mała dawka
        elif glucose_diff_max > 50:
            dose_feedback = 1  # Za mała dawka
        elif glucose_diff_min < -100:
            dose_feedback = -2  # Znacznie za duża dawka
        elif glucose_diff_min < -50:
            dose_feedback = -1  # Za duża dawka
        else:
            dose_feedback = 0

        merged_data.append([
            row['glucose'], row['insulin'], WW, hour,
            glucose_diff_max, glucose_diff_min, dose_feedback
        ])

# Tworzymy DataFrame z połączonych danych
data = pd.DataFrame(merged_data, columns=[
    'glucose', 'insulin', 'WW', 'hour',
    'glucose_diff_max', 'glucose_diff_min', 'dose_feedback'
])

# ✅ Osobne skalery dla każdej zmiennej → StandardScaler
scalers = {}
for col in data.columns[:-1]:
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[[col]])
    scalers[col] = scaler

# ✅ Klasyfikacja z RandomForest
X = data.iloc[:, :-1].values
y = data['dose_feedback'].values

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X, y)

# ==== TESTY ====
test_input = pd.DataFrame([[150, 5, 5, 9.5, 30, -70]], columns=data.columns[:-1])

# ✅ Skalowanie testów zgodnie z zakresem danych treningowych
for col in test_input.columns:
    test_input[col] = scalers[col].transform(test_input[[col]])

# ✅ Predykcja
predicted_class = clf.predict(test_input)[0]

# ✅ Interpretacja wyniku
if predicted_class == 2:
    print("🔥 Dawka znacznie za mała → Zwiększ dawkę insuliny!")
elif predicted_class == 1:
    print("🔥 Dawka za mała → Zwiększ dawkę insuliny!")
elif predicted_class == -2:
    print("❄️ Dawka znacznie za duża → Zmniejsz dawkę insuliny!")
elif predicted_class == -1:
    print("❄️ Dawka za duża → Zmniejsz dawkę insuliny!")
else:
    print("✅ Dawka OK → Kontynuuj w tej samej dawce.")

# ✅ Raport z klasyfikacji
y_pred = clf.predict(X)
print(classification_report(y, y_pred))


❄️ Dawka za duża → Zmniejsz dawkę insuliny!
              precision    recall  f1-score   support

          -2       1.00      1.00      1.00         3
          -1       1.00      1.00      1.00         9
           0       1.00      1.00      1.00        43
           1       1.00      1.00      1.00        32
           2       1.00      1.00      1.00         4

    accuracy                           1.00        91
   macro avg       1.00      1.00      1.00        91
weighted avg       1.00      1.00      1.00        91



C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import json

# ==== Wczytywanie pliku CSV ====
glucose_data = pd.read_csv('Michał._glucose_8-3-2025.csv', skiprows=1, delimiter=',', low_memory=False)

# Konwersja daty na format datetime
glucose_data['Device Timestamp'] = pd.to_datetime(glucose_data['Device Timestamp'], format='%d-%m-%Y %H:%M', errors='coerce')

# Usuwamy błędne daty
glucose_data = glucose_data.dropna(subset=['Device Timestamp'])

# ✅ Łączymy dane z dwóch kolumn (Historic + Scan)
glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(glucose_data['Scan Glucose mg/dL'])

glucose_data = glucose_data.rename(columns={'Device Timestamp': 'date'})
glucose_data = glucose_data[['date', 'glucose']]

# Usuwamy przypadki, gdzie poziom glukozy wynosi 0
glucose_data = glucose_data[glucose_data['glucose'] > 0]

# ==== Wczytywanie pliku JSON ====
with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

# Przetwarzanie danych JSON
processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator:
            glucose = calculator.get('glucose')
            insulin = calculator.get('units', {}).get('short')
            date = pd.to_datetime(calculator.get('date'), errors='coerce')
            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))
            processed_data.append([date, glucose, insulin, carbs])

meals_df = pd.DataFrame(processed_data, columns=['date', 'glucose', 'insulin', 'carbs'])
meals_df['date'] = meals_df['date'].dt.tz_convert(None)

# ==== Dopasowywanie danych ====
glucose_data = glucose_data.sort_values('date').reset_index(drop=True)
meals_df = meals_df.sort_values('date').reset_index(drop=True)

merged_data = []
for index, row in meals_df.iterrows():
    meal_time = row['date']
    glucose_window = glucose_data[(glucose_data['date'] >= meal_time + pd.Timedelta(hours=1)) & (glucose_data['date'] <= meal_time + pd.Timedelta(hours=2.5))]

    if not glucose_window.empty:
        glucose_max = np.nanmax(glucose_window['glucose'].values)
        glucose_min = np.nanmin(glucose_window['glucose'].values)

        WW = row['carbs'] / 10
        insulin_per_WW = row['insulin'] / WW if WW > 0 else 0

        hour = meal_time.hour + meal_time.minute / 60

        if 6 <= meal_time.hour < 12:
            part_of_day = 0  # Poranek
        elif 12 <= meal_time.hour < 18:
            part_of_day = 1  # Popołudnie
        elif 18 <= meal_time.hour < 24:
            part_of_day = 2  # Wieczór
        else:
            part_of_day = 3  # Noc

        glucose_diff_max = glucose_max - row['glucose']
        glucose_diff_min = glucose_min - row['glucose']

        # ✅ Ulepszona logika klasyfikacji
        if insulin_per_WW > 3.0:
            dose_feedback = -2  # Znacznie za duża dawka
        elif insulin_per_WW > 2.5:
            dose_feedback = -1  # Za duża dawka
        elif (80 <= row['glucose'] + glucose_diff_max <= 150) and (80 <= row['glucose'] + glucose_diff_min <= 150):
            dose_feedback = 0  # OK
        elif glucose_diff_max > 150:
            dose_feedback = 2  # Znacznie za mała dawka
        elif glucose_diff_max > 75:
            dose_feedback = 1  # Za mała dawka
        elif glucose_diff_min < -150:
            dose_feedback = -2  # Znacznie za duża dawka
        elif glucose_diff_min < -75:
            dose_feedback = -1  # Za duża dawka
        else:
            dose_feedback = 0

        merged_data.append([
            row['glucose'], row['insulin'], WW, hour,
            part_of_day, insulin_per_WW, dose_feedback
        ])

# Tworzymy DataFrame z połączonych danych
data = pd.DataFrame(merged_data, columns=[
    'glucose', 'insulin', 'WW', 'hour', 'part_of_day', 'insulin_per_WW', 'dose_feedback'
])

# ✅ Osobne skalery dla każdej zmiennej → StandardScaler
scalers = {}
for col in data.columns[:-1]:
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[[col]])
    scalers[col] = scaler

# ✅ Klasyfikacja z RandomForest
X = data[['glucose', 'insulin', 'WW', 'hour', 'part_of_day', 'insulin_per_WW']].values
y = data['dose_feedback'].values

clf = RandomForestClassifier(n_estimators=700, max_depth=15, random_state=42)
clf.fit(X, y)

# ==== TESTY ====
test_cases = [
    [150, 5, 5, 9.5, 1, 5],
    [100, 5, 3, 12.0, 0, 1.7],
    [200, 15, 6, 18.5, 2, 2.5],
    [90, 2, 2, 22.0, 3, 1.0],
    [180, 25, 5, 8.0, 1, 5.0],
    [140, 6, 4, 14.0, 2, 1.5]
]

test_df = pd.DataFrame(test_cases, columns=['glucose', 'insulin', 'WW', 'hour', 'part_of_day', 'insulin_per_WW'])
for col in test_df.columns:
    test_df[col] = scalers[col].transform(test_df[[col]])

predictions = clf.predict(test_df)

for i, pred in enumerate(predictions):
    print(f"Test {i+1}: {test_cases[i]} -> Klasyfikacja: {pred}")


Test 1: [150, 5, 5, 9.5, 1, 5] -> Klasyfikacja: 1
Test 2: [100, 5, 3, 12.0, 0, 1.7] -> Klasyfikacja: 1
Test 3: [200, 15, 6, 18.5, 2, 2.5] -> Klasyfikacja: 0
Test 4: [90, 2, 2, 22.0, 3, 1.0] -> Klasyfikacja: 2
Test 5: [180, 25, 5, 8.0, 1, 5.0] -> Klasyfikacja: 0
Test 6: [140, 6, 4, 14.0, 2, 1.5] -> Klasyfikacja: 0


C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [15]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import json

# ==== Wczytywanie pliku CSV ====
glucose_data = pd.read_csv('Michał._glucose_8-3-2025.csv', skiprows=1, delimiter=',', low_memory=False)

# Konwersja daty na format datetime
glucose_data['Device Timestamp'] = pd.to_datetime(glucose_data['Device Timestamp'], format='%d-%m-%Y %H:%M', errors='coerce')

# Usuwamy błędne daty
glucose_data = glucose_data.dropna(subset=['Device Timestamp'])

# ✅ Łączymy dane z dwóch kolumn (Historic + Scan)
glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(glucose_data['Scan Glucose mg/dL'])

glucose_data = glucose_data.rename(columns={'Device Timestamp': 'date'})
glucose_data = glucose_data[['date', 'glucose']]

# Usuwamy przypadki, gdzie poziom glukozy wynosi 0
glucose_data = glucose_data[glucose_data['glucose'] > 0]

# ==== Wczytywanie pliku JSON ====
with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

# Przetwarzanie danych JSON
processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator:
            glucose = calculator.get('glucose')
            insulin = calculator.get('units', {}).get('short')
            date = pd.to_datetime(calculator.get('date'), errors='coerce')
            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))
            processed_data.append([date, glucose, insulin, carbs])

meals_df = pd.DataFrame(processed_data, columns=['date', 'glucose', 'insulin', 'carbs'])
meals_df['date'] = meals_df['date'].dt.tz_convert(None)

# ==== Dopasowywanie danych ====
glucose_data = glucose_data.sort_values('date').reset_index(drop=True)
meals_df = meals_df.sort_values('date').reset_index(drop=True)

merged_data = []
for index, row in meals_df.iterrows():
    meal_time = row['date']
    glucose_window = glucose_data[(glucose_data['date'] >= meal_time + pd.Timedelta(hours=1)) & (glucose_data['date'] <= meal_time + pd.Timedelta(hours=2.5))]

    if not glucose_window.empty:
        glucose_max = np.nanmax(glucose_window['glucose'].values)
        glucose_min = np.nanmin(glucose_window['glucose'].values)
        glucose_values = glucose_window['glucose'].values.tolist()

        WW = row['carbs'] / 10
        insulin_per_WW = row['insulin'] / WW if WW > 0 else 0

        hour = meal_time.hour + meal_time.minute / 60

        if 6 <= meal_time.hour < 12:
            part_of_day = 0  # Poranek
        elif 12 <= meal_time.hour < 18:
            part_of_day = 1  # Popołudnie
        elif 18 <= meal_time.hour < 24:
            part_of_day = 2  # Wieczór
        else:
            part_of_day = 3  # Noc

        glucose_diff_max = glucose_max - row['glucose']
        glucose_diff_min = glucose_min - row['glucose']

        # ✅ Nowa logika klasyfikacji
        if (80 <= row['glucose'] + glucose_diff_max <= 150) and (80 <= row['glucose'] + glucose_diff_min <= 150):
            dose_feedback = 0  # OK
        elif row['glucose'] > 150 and glucose_diff_max > 30:
            dose_feedback = 1  # Za mała dawka
        elif row['glucose'] < 80 and glucose_diff_min < -30:
            dose_feedback = -1  # Za duża dawka
        elif glucose_diff_max > 150:
            dose_feedback = 2  # Znacznie za mała dawka
        elif glucose_diff_max > 75:
            dose_feedback = 1  # Za mała dawka
        elif glucose_diff_min < -150:
            dose_feedback = -2  # Znacznie za duża dawka
        elif glucose_diff_min < -75:
            dose_feedback = -1  # Za duża dawka
        else:
            dose_feedback = 0

        merged_data.append([
            row['glucose'], row['insulin'], WW, hour,
            part_of_day, dose_feedback, glucose_values
        ])

# Tworzymy DataFrame z połączonych danych
data = pd.DataFrame(merged_data, columns=[
    'glucose', 'insulin', 'WW', 'hour', 'part_of_day', 'dose_feedback', 'glucose_values'
])

print(data)

# ✅ Osobne skalery dla każdej zmiennej → StandardScaler
scalers = {}
for col in data.columns[:-2]:
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[[col]])
    scalers[col] = scaler

# ✅ Klasyfikacja z RandomForest
X = data[['glucose', 'insulin', 'WW', 'hour', 'part_of_day']].values
y = data['dose_feedback'].values

clf = RandomForestClassifier(n_estimators=700, max_depth=15, random_state=42)
clf.fit(X, y)

# ==== TESTY ====
test_cases = [
    [150, 2, 5, 9.5, 1],
    [100, 5, 3, 12.0, 0],
    [200, 15, 6, 18.5, 2],
    [90, 2, 2, 22.0, 3],
    [180, 25, 5, 8.0, 1],
    [140, 6, 4, 14.0, 2]
]

test_df = pd.DataFrame(test_cases, columns=['glucose', 'insulin', 'WW', 'hour', 'part_of_day'])
for col in test_df.columns:
    test_df[col] = scalers[col].transform(test_df[[col]])

predictions = clf.predict(test_df)

for i, pred in enumerate(predictions):
    print(f"Test {i+1}: {test_cases[i]} -> Klasyfikacja: {pred}")


    glucose  insulin    WW       hour  part_of_day  dose_feedback   
0       156      6.1  6.06   8.516667            0              0  \
1       143      3.2  3.21  11.500000            0              0   
2       197      1.8  1.50   9.016667            0             -1   
3       124      5.0  5.03  23.600000            2              1   
4       152      4.0  5.75  19.800000            2              1   
..      ...      ...   ...        ...          ...            ...   
86      161      2.2  1.04   8.883333            0              0   
87      123      3.5  3.50  11.200000            0              0   
88      109      3.5  3.00  13.083333            1              0   
89      170      2.5  1.50   9.850000            0              0   
90      150      9.0  9.69  16.516667            1              0   

                                       glucose_values  
0   [127.0, 129.0, 134.0, 136.0, 132.0, 135.0, 136...  
1   [122.0, 119.0, 108.0, 102.0, 98.0, 81.0, 97.0,...  
2  

C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [30]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
import json

# ==== Wczytywanie pliku CSV ====
glucose_data = pd.read_csv('Michał._glucose_8-3-2025.csv', skiprows=1, delimiter=',', low_memory=False)

glucose_data['Device Timestamp'] = pd.to_datetime(glucose_data['Device Timestamp'], format='%d-%m-%Y %H:%M', errors='coerce')
glucose_data = glucose_data.dropna(subset=['Device Timestamp'])

glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(glucose_data['Scan Glucose mg/dL'])
glucose_data = glucose_data.rename(columns={'Device Timestamp': 'date'})
glucose_data = glucose_data[['date', 'glucose']]
glucose_data = glucose_data[glucose_data['glucose'] > 0]

# ==== Wczytywanie pliku JSON ====
with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator:
            glucose = calculator.get('glucose')
            insulin = calculator.get('units', {}).get('short')
            date = pd.to_datetime(calculator.get('date'), errors='coerce')
            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))
            processed_data.append([date, glucose, insulin, carbs])

meals_df = pd.DataFrame(processed_data, columns=['date', 'glucose', 'insulin', 'carbs'])
meals_df['date'] = meals_df['date'].dt.tz_convert(None)

glucose_data = glucose_data.sort_values('date').reset_index(drop=True)
meals_df = meals_df.sort_values('date').reset_index(drop=True)

merged_data = []
for index, row in meals_df.iterrows():
    meal_time = row['date']
    glucose_window = glucose_data[(glucose_data['date'] >= meal_time + pd.Timedelta(hours=1)) & (glucose_data['date'] <= meal_time + pd.Timedelta(hours=2.5))]

    if not glucose_window.empty:
        glucose_max = np.nanmax(glucose_window['glucose'].values)
        glucose_min = np.nanmin(glucose_window['glucose'].values)

        WW = row['carbs'] / 10
        insulin_per_WW = row['insulin'] / WW if WW > 0 else 0
        hour = meal_time.hour + meal_time.minute / 60
        part_of_day = 0 if 6 <= meal_time.hour < 12 else 1 if 12 <= meal_time.hour < 18 else 2 if 18 <= meal_time.hour < 24 else 3
        glucose_diff_max = glucose_max - row['glucose']
        glucose_diff_min = glucose_min - row['glucose']

        if (80 <= row['glucose'] + glucose_diff_max <= 150) and (80 <= row['glucose'] + glucose_diff_min <= 150):
            dose_feedback = 0  # OK
        elif glucose_diff_max > 150:
            dose_feedback = 2  # Znacznie za mała dawka
        elif glucose_diff_max > 75:
            dose_feedback = 1  # Za mała dawka
        elif glucose_diff_min < -150:
            dose_feedback = -2  # Znacznie za duża dawka
        elif glucose_diff_min < -75:
            dose_feedback = -1  # Za duża dawka
        else:
            dose_feedback = 0

        merged_data.append([row['glucose'], row['insulin'], WW, hour, part_of_day, dose_feedback])

data = pd.DataFrame(merged_data, columns=['glucose', 'insulin', 'WW', 'hour', 'part_of_day', 'dose_feedback'])
scalers = {}
for col in data.columns[:-1]:
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[[col]])
    scalers[col] = scaler

y = data['dose_feedback'].values + 2  # Przesunięcie klas do zakresu [0, 1, 2, 3, 4]
y = y - np.min(y)
X = data[['glucose', 'insulin', 'WW', 'hour', 'part_of_day']].values

clf = xgb.XGBClassifier(objective='multi:softmax', num_class=5, n_estimators=700, max_depth=15, random_state=42, use_label_encoder=False, eval_metric='mlogloss')
clf.fit(X, y)

test_cases = [
    [150, 2, 5, 9.5, 1],
    [100, 5, 3, 12.0, 0],
    [200, 15, 6, 18.5, 2],
    [90, 2, 2, 22.0, 3],
    [180, 25, 5, 8.0, 1],
    [140, 6, 4, 14.0, 2]
]

test_df = pd.DataFrame(test_cases, columns=['glucose', 'insulin', 'WW', 'hour', 'part_of_day'])
for col in test_df.columns:
    test_df[col] = scalers[col].transform(test_df[[col]])

predictions = clf.predict(test_df) - 2  # Powrót do oryginalnych etykiet
for i, pred in enumerate(predictions):
    print(f"Test {i+1}: {test_cases[i]} -> Klasyfikacja: {pred}")


C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\core.py:158: UserWarning: [00:41:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Test 1: [150, 2, 5, 9.5, 1] -> Klasyfikacja: -1
Test 2: [100, 5, 3, 12.0, 0] -> Klasyfikacja: -1
Test 3: [200, 15, 6, 18.5, 2] -> Klasyfikacja: -1
Test 4: [90, 2, 2, 22.0, 3] -> Klasyfikacja: 1
Test 5: [180, 25, 5, 8.0, 1] -> Klasyfikacja: -1
Test 6: [140, 6, 4, 14.0, 2] -> Klasyfikacja: -1
